In [ ]:
# default_exp models.utils

In [ ]:
# hide
%load_ext autoreload
%autoreload 2

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export
from grade_classif.core import ifnone
from grade_classif.models.hooks import Hooks
from grade_classif.imports import *

In [ ]:
# export
def register_model(fn):
    # lookup containing module
    mod = sys.modules[fn.__module__]
    module_name_split = fn.__module__.split('.')
    module_name = module_name_split[-1] if len(module_name_split) else ''

    # add model to __all__ in module
    model_name = fn.__name__
    if hasattr(mod, '__all__'):
        mod.__all__.append(model_name)
    else:
        mod.__all__ = [model_name]
    return fn

In [ ]:
# export
def named_leaf_modules(name, model):
    named_children = list(model.named_children())
    if named_children==[]:
        model.name = name
        return [model]
    else:
        res = []
        for n, m in named_children:
            pref = name+'.' if name != '' else ''
            res += named_leaf_modules(pref+n, m)
        return res

In [ ]:
# export
def get_sizes(model, input_shape=(3, 224, 224), leaf_modules=None):     
    leaf_modules = ifnone(leaf_modules, named_leaf_modules('', model))
    
    class Count:
        def __init__(self):
            self.k = 0
    count = Count()        
    def _hook(model, input, output):
        model.k = count.k
        count.k += 1
        return model, output
    
    with Hooks(leaf_modules, _hook) as hooks:
        x = torch.rand(2, *input_shape)
        model.eval()(x)
        sizes = [hook.stored[1].shape for hook in hooks]
        mods = [hook.stored[0] for hook in hooks]
    idxs = np.argsort([mod.k for mod in mods])
    return np.array(sizes)[idxs], np.array(mods)[idxs]

In [ ]:
# export
def gaussian_mask(m, s, d, R, C):
    # indices to create centres
    R = torch.arange(R, dtype=torch.float32).reshape((R, 1))
    C = torch.arange(C, dtype=torch.float32).reshape((1, C))
    centres = m + R * d
    column_centres = C - centres
    mask = torch.exp(-.5 * torch.square(column_centres / s))
    # we add eps for numerical stability
    normalised_mask = mask / (mask.sum(1, keepdims=True) + 1e-8)
    return normalised_mask

In [ ]:
# export
def get_num_features(model):
    sizes, _ = get_sizes(model)
    return sizes[-1, 1]

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_train.ipynb.
Converted 02_predict.ipynb.
Converted 10_data.read.ipynb.
Converted 11_data.loaders.ipynb.
Converted 12_data.dataset.ipynb.
Converted 13_data.utils.ipynb.
Converted 14_data.transforms.ipynb.
Converted 20_models.plmodules.ipynb.
Converted 21_models.modules.ipynb.
Converted 22_models.utils.ipynb.
Converted 23_models.hooks.ipynb.
Converted 24_models.metrics.ipynb.
Converted 25_models.losses.ipynb.
Converted 80_params.defaults.ipynb.
Converted 81_params.parser.ipynb.
Converted 99_index.ipynb.
